# Detección de Fraude Financiero - Análisis Exploratorio
## Usando Topología y Grafos Dirigidos

Este notebook demuestra el uso de la biblioteca para detectar fraude en transacciones financieras.

In [ ]:
# Importar librerías necesarias
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Importar módulos del proyecto
from src.data_processing import TransactionDataProcessor, generate_synthetic_transactions
from src.graph_construction import TransactionGraph
from src.topology_analysis import TopologyAnalyzer, analyze_graph_topology
from src.anomaly_detection import AnomalyDetector, detect_fraud_patterns
from src.visualization import GraphVisualizer

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Generación de Datos Sintéticos

Generamos un dataset sintético de transacciones financieras que incluye transacciones normales y fraudulentas.

In [ ]:
# Generar datos sintéticos
transactions = generate_synthetic_transactions(
    n_accounts=100,
    n_transactions=1500,
    fraud_ratio=0.05,
    random_state=42
)

print(f"Total de transacciones: {len(transactions)}")
print(f"Transacciones fraudulentas: {transactions['is_fraud'].sum()} ({transactions['is_fraud'].mean():.2%})")
print(f"\nPrimeras transacciones:")
transactions.head(10)

In [ ]:
# Estadísticas básicas
print("Estadísticas de montos de transacción:")
print(transactions['amount'].describe())

# Comparar transacciones normales vs fraudulentas
print("\nMonto promedio por tipo:")
print(transactions.groupby('is_fraud')['amount'].agg(['mean', 'median', 'std']))

## 2. Construcción del Grafo de Transacciones

Construimos un grafo dirigido donde los nodos son cuentas y las aristas son transacciones.

In [ ]:
# Construir el grafo
tg = TransactionGraph(directed=True)
tg.load_data(transactions)
graph = tg.build()

# Obtener estadísticas del grafo
stats = tg.get_basic_stats()
print("Estadísticas del Grafo:")
for key, value in stats.items():
    print(f"  {key}: {value}")

## 3. Análisis Topológico

Analizamos las propiedades topológicas del grafo de transacciones.

In [ ]:
# Crear analizador topológico
analyzer = TopologyAnalyzer(graph)

# Calcular medidas de centralidad
print("Calculando medidas de centralidad...")
centrality = analyzer.compute_centrality_measures()

# Mostrar nodos más importantes por PageRank
if 'pagerank' in centrality and centrality['pagerank']:
    print("\nTop 10 nodos por PageRank:")
    sorted_pagerank = sorted(centrality['pagerank'].items(), key=lambda x: x[1], reverse=True)[:10]
    for node, score in sorted_pagerank:
        print(f"  {node}: {score:.6f}")

In [ ]:
# Detectar comunidades
print("Detectando comunidades...")
communities = analyzer.detect_communities(method='label_propagation')
n_communities = len(set(communities.values()))
print(f"Número de comunidades detectadas: {n_communities}")

# Distribución de tamaños de comunidades
community_sizes = pd.Series(communities.values()).value_counts()
print("\nTamaños de comunidades:")
print(community_sizes.head(10))

In [ ]:
# Identificar hubs (nodos con alta conectividad)
hubs = analyzer.identify_hubs(threshold_percentile=90)
print(f"\nHubs identificados (top 10%): {len(hubs)}")
print(f"Ejemplos: {hubs[:10]}")

## 4. Detección de Anomalías

Aplicamos algoritmos de detección de anomalías para identificar transacciones y cuentas sospechosas.

In [ ]:
# Crear detector de anomalías
detector = AnomalyDetector(graph, contamination=0.05)

# Detectar anomalías en nodos
print("Detectando anomalías...")
anomaly_scores = detector.detect_node_anomalies(method='isolation_forest')

# Obtener top anomalías
top_anomalies = detector.get_top_anomalies(n=10)
print("\nTop 10 nodos anómalos:")
for node, score in top_anomalies:
    print(f"  {node}: score={score:.4f}")

In [ ]:
# Detectar patrones anómalos
patterns = detector.detect_anomalous_patterns()

print("Patrones anómalos detectados:")
print(f"  Nodos aislados: {len(patterns['isolated_nodes'])}")
print(f"  Patrones estrella: {len(patterns['star_patterns'])}")
print(f"  Ciclos anómalos: {len(patterns['anomalous_cycles'])}")

if patterns['star_patterns']:
    print(f"\n  Ejemplos de patrones estrella: {patterns['star_patterns'][:5]}")

In [ ]:
# Explicar por qué un nodo es anómalo
if top_anomalies:
    anomalous_node = top_anomalies[0][0]
    explanation = detector.explain_anomaly(anomalous_node)
    
    print(f"\nExplicación para el nodo más anómalo ({anomalous_node}):")
    print(f"  Score de anomalía: {explanation['anomaly_score']:.4f}")
    print("\n  Características:")
    for feature, details in explanation['comparisons'].items():
        if details['is_outlier']:
            print(f"    {feature}: valor={details['value']:.2f}, z-score={details['z_score']:.2f} (OUTLIER)")

## 5. Visualización de Resultados

Visualizamos el grafo y los resultados de la detección de anomalías.

In [ ]:
# Crear visualizador
visualizer = GraphVisualizer(graph, figsize=(14, 10))

# Visualizar el grafo con scores de anomalía
fig = visualizer.plot_graph(
    node_colors=anomaly_scores,
    layout='spring',
    title='Grafo de Transacciones (color indica score de anomalía)'
)
plt.show()

In [ ]:
# Graficar scores de anomalía
fig = visualizer.plot_anomaly_scores(anomaly_scores, top_n=20)
plt.show()

In [ ]:
# Distribución de grados
fig = visualizer.plot_degree_distribution()
plt.show()

In [ ]:
# Comparar medidas de centralidad
centrality_subset = {
    'degree': centrality.get('degree'),
    'pagerank': centrality.get('pagerank'),
    'betweenness': centrality.get('betweenness')
}
centrality_subset = {k: v for k, v in centrality_subset.items() if v is not None}

if centrality_subset:
    fig = visualizer.plot_centrality_comparison(centrality_subset, top_n=10)
    plt.show()

In [ ]:
# Visualizar estructura de comunidades
fig = visualizer.plot_community_structure(communities, layout='spring')
plt.show()

In [ ]:
# Timeline de transacciones
fig = visualizer.plot_transaction_timeline(transactions)
plt.show()

## 6. Evaluación y Conclusiones

Evaluamos qué tan bien nuestro método detecta fraudes.

In [ ]:
# Crear un DataFrame con resultados
results_df = transactions[['transaction_id', 'source', 'target', 'amount', 'is_fraud']].copy()

# Agregar scores de anomalía para fuentes
results_df['source_anomaly_score'] = results_df['source'].map(anomaly_scores)
results_df['target_anomaly_score'] = results_df['target'].map(anomaly_scores)

# Determinar si es anómalo (umbral en percentil 10)
threshold = np.percentile(list(anomaly_scores.values()), 10)
results_df['is_anomalous'] = (
    (results_df['source_anomaly_score'] < threshold) | 
    (results_df['target_anomaly_score'] < threshold)
)

# Matriz de confusión simple
from sklearn.metrics import confusion_matrix, classification_report

print("\nMatriz de Confusión:")
print(confusion_matrix(results_df['is_fraud'], results_df['is_anomalous']))

print("\nReporte de Clasificación:")
print(classification_report(results_df['is_fraud'], results_df['is_anomalous'], 
                          target_names=['Normal', 'Fraude']))

## 7. Exportar Resultados

In [ ]:
# Guardar resultados
import os
os.makedirs('results', exist_ok=True)

# Exportar grafo
tg.export_graph('results/transaction_graph.gexf', format='gexf')
print("Grafo exportado a: results/transaction_graph.gexf")

# Exportar resultados de detección
results_df.to_csv('results/detection_results.csv', index=False)
print("Resultados exportados a: results/detection_results.csv")

# Exportar anomalías detectadas
anomaly_df = pd.DataFrame([
    {'node': node, 'anomaly_score': score}
    for node, score in sorted(anomaly_scores.items(), key=lambda x: x[1])
])
anomaly_df.to_csv('results/anomaly_scores.csv', index=False)
print("Scores de anomalía exportados a: results/anomaly_scores.csv")

print("\n¡Análisis completado!")